<a href="https://colab.research.google.com/github/followthefourleafedclover/AnonDiscord-/blob/main/cupid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install discord.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.3 MB/s eta 0:00:00


In [ ]:
pip install PyNaCl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 9.2 MB/s eta 0:00:00


In [ ]:
import discord
from discord.ext import commands
import nest_asyncio
nest_asyncio.apply()
import pandas as pd
import time
import asyncio

In [ ]:
class MatchMakerCSV:
  def __init__(self):
    pass

  def ImportCSV(self, path_name):
    self.df  = pd.read_csv(path_name)

    # Assume data is in catagories as cols and rows as responses

    self.rows, self.cols = self.df.shape

    self.cata = [x.lower() for x in self.df.columns]


  def Minimum_Age(self, age, meathod):
    age = int(age)
    if meathod == 0:
      return [(age / 2) + 7, 2*(age-7)]

    if meathod == 1:
      return [age - 2, age + 2]

    return [0, age];



  def Find_Clusters(self, person):

    row_values = self.df.iloc[person]
    age_index = self.cata.index('age')

    #Age #1
    self.age = row_values[self.df.columns[age_index]]

    if self.age == 14:
      min_age  = self.Minimum_Age(self.age, 1)[0]
    else:
      min_age  = self.Minimum_Age(self.age, 2)[0]

    self.cluster = self.df[(self.df[self.cata[age_index]] >= min_age)]
    self.cluster = self.cluster.drop(person)

      # likes and dislikes
    self.cluster_array = []
    for x in range(self.cluster.shape[0]):
      i = []
      for y in range(len(self.cata)):
        i.append(self.cluster.iloc[x][y])
      self.cluster_array.append(i)

    likes_index = self.cata.index('likes')
    dislikes_index = self.cata.index('dislikes')

    self.likes = []
    self.dislikes  = []

    self.person_likes = row_values[self.cata.index('likes')].split(',')
    self.person_dislikes = row_values[self.cata.index('dislikes')].split(',')

    for item in self.cluster_array:
      self.likes.append([x.strip() for x in item[likes_index].split(',')])
      self.dislikes.append([x.strip() for x in item[dislikes_index].split(',')])

      # intersection algorithim
    self.l_scores  = []

    a = set(self.person_likes)
    for item in self.likes:
      b = set(item)
      self.l_scores.append(len(a.intersection(b)))

    self.d_scores = []

    c = set(self.person_dislikes)
    for item in self.likes:
      d = set(item)
      self.d_scores.append(len(c.intersection(d)))


    self.scores  = [x - y for x, y in zip(self.l_scores, self.d_scores)]

    max_indexes  = []

    self.max  = max(self.scores)

    while self.max in self.scores:
      max_indexes.append(self.scores.index(self.max))
      self.scores.remove(self.max)

    for person in max_indexes:
      if self.max <= 0:
        return "You have no matches"
      else:
        return f"These are your best matches \n{self.cluster.iloc[person]}"
    return ";)"






In [ ]:
class MatchMakerRoles:
  def __init__(self):
    pass



  def Minimum_Age(self, age, meathod):
    age = int(age)
    if meathod == 0:
      return [(age / 2) + 7, 2*(age-7)]

    if meathod == 1:
      return [age - 2, age + 2]

    return [0, age];



  def Find_Clusters(self, interaction, preferences):

    self.personal_roles = []
    for role in interaction.user.roles:
      self.personal_roles.append(role.name)
    self.personal_roles = self.personal_roles[1:]

    self.members_list = []

    for member in interaction.guild.members:
      if member.id != interaction.user.id:
        self.members_list.append(member)



    # last part

    self.roles = []
    for index, member in enumerate(self.members_list):
      place_holder = []
      for role in self.members_list[index].roles:
        place_holder.append(role.name)

      place_holder = place_holder[1:]
      self.roles.append(place_holder)


    # gender filter

    #cluster
    self.members_roles = list(zip(self.members_list, self.roles))
    self.del_indexes = []


    if preferences[0] != 'any':
      for index, item in enumerate(self.members_roles):
          if not (preferences[0] in item[1]):
            self.del_indexes.append(index)


    for index in sorted(self.del_indexes, reverse=True):
      del self.members_roles[index]

    if len(self.members_roles) > 0:

      self.like_scores = []

      self.personal_roles_set = set(self.personal_roles)

      for cache in self.members_roles:
        t_set  = set(cache[1])
        self.like_scores.append(len(self.personal_roles_set.intersection(t_set)))

      self.max_indexes = []
      self.max = max(self.like_scores)
      self.like_scores_backup = self.like_scores.copy()

      if not self.max <= 0:

        for index, item in enumerate(self.like_scores):
          if item == self.max:
            self.max_indexes.append(index)

        self.matches = []
        self.match_per = []

        self.match_members = []

        for index in self.max_indexes:
          self.matches.append(self.members_roles[index])


        for index, item in enumerate(self.max_indexes):
          self.match_per.append(round((self.max/len(self.matches[index][1]))*100, 2))
          self.match_members.append(self.members_roles[item][0])

        return self.match_members, self.match_per

      return [], [0]
    return [], [0]




In [ ]:
m1 = MatchMakerCSV()
m2 = MatchMakerRoles()


In [ ]:
from ast import Delete
import requests
import typing


def run():
  client = commands.Bot(command_prefix="/", intents=discord.Intents.all())

  @client.event
  async def on_ready():
    print("ready")

  class RoleEngine():
    def __init__(self):
      pass

    async def start(self, ctx):

      self.members = []

      for member in ctx.guild.members:
        self.members.append(member)

      likes = []
      dislikes = []

      for i in range(len(self.members)):
        placeholder1 = []
        placeholder2 = []
        for role in self.members[i].roles:
          if 'l:' in role.name:
            placeholder1.append(role.name)

          if 'd:' in role.name:
            placeholder2.append(role.name)
        likes.append(placeholder1)
        dislikes.append(placeholder2)


      return list(zip(self.members, likes, dislikes))





  class Angel(discord.ui.View):
    def __init__(self):
      super().__init__(timeout=None)



    async def send(self, ctx):

      self.r = RoleEngine()
      self.attrs = await self.r.start(ctx)

      self.members = []

      for member in ctx.guild.members:
        self.members.append(member)


      self.index = 0

      self.title = '𝐄𝐧𝐨𝐜𝐡𝐢𝐚𝐧 ଘ(੭ˊᵕˋ)੭* ੈ✩‧˚'

      self.desc = '𝐒𝐰𝐢𝐩𝐞 𝐭𝐡𝐫𝐨𝐮𝐠𝐡 𝐮𝐬𝐞𝐫𝐬 𝐨𝐧 𝐭𝐡𝐞 𝐬𝐞𝐫𝐯𝐞𝐫, 𝐥𝐢𝐤𝐞 𝐨𝐫 𝐝𝐢𝐬𝐥𝐢𝐤𝐞 𝐭𝐡𝐞𝐦 𝐛𝐲 𝐮𝐬𝐢𝐧𝐠 𝐭𝐡𝐞 𝐛𝐮𝐭𝐭𝐨𝐧𝐬 𝐛𝐞𝐥𝐨𝐰.\n𝐈𝐟 𝐭𝐡𝐞𝐲 𝐥𝐢𝐤𝐞 𝐲𝐨𝐮 𝐭𝐨𝐨, 𝐭𝐡𝐞𝐧 𝐰𝐞 𝐰𝐢𝐥𝐥 𝐧𝐨𝐭𝐢𝐟𝐲 𝐲𝐨𝐮 𝐨𝐟 𝐚 𝐦𝐚𝐭𝐜𝐡. 𝐆𝐨𝐨𝐝 𝐥𝐮𝐜𝐤!'

      self.ll = [0 for x in range(len(self.attrs))]
      self.dd = [0 for x in range(len(self.attrs))]

      self.liked = [False for x in range(len(self.attrs))]
      self.disliked = [False for x in range(len(self.attrs))]


      embed = discord.Embed(title=self.title, description=self.desc, color=discord.Color.fuchsia())
      #embed.add_field(name=' ', value='*✭˚･ﾟ✧*･ﾟ*✭˚･ﾟ✧*･ﾟ* *✭˚･ﾟ✧*･ﾟ*✭˚･ﾟ✧*･ﾟ* *✭˚･ﾟ✧*･ﾟ*✭˚･ﾟ✧*･ﾟ*')
      embed.set_thumbnail(url=self.attrs[self.index][0].display_avatar.url)
      embed.add_field(name="Name", value=f'{self.attrs[self.index][0].global_name}')
      likes = [x.split('l:')[-1] for x in self.attrs[self.index][1]]
      likes = ', '.join(likes)
      embed.add_field(name='𝐋𝐢𝐤𝐞𝐬', value=f'{likes}')
      dislikes = [x.split('d:')[-1] for x in self.attrs[self.index][2]]
      dislikes = ', '.join(dislikes)
      embed.add_field(name='𝐃𝐢𝐬𝐥𝐢𝐤𝐞𝐬', value=f'{dislikes}')
      embed.add_field(name='like count', value=f'{self.ll[self.index]}')
      embed.add_field(name='dislike count', value=f'{self.dd[self.index]}')
      self.message = await ctx.send(embed=embed, view=self)


    async def update_attrs(self, mld):
      self.attrs.append(mld)
      self.ll.append(0)
      self.dd.append(0)
      self.liked.append(False)
      self.disliked.append(False)



    async def update(self):
      embed = discord.Embed(title=self.title, description=self.desc, color=discord.Color.fuchsia())
      embed.set_thumbnail(url=self.attrs[self.index][0].display_avatar.url)
      embed.add_field(name="Name", value=f'{self.attrs[self.index][0].global_name}')
      likes = [x.split('l:')[-1] for x in self.attrs[self.index][1]]
      likes = ', '.join(likes)
      embed.add_field(name='𝐋𝐢𝐤𝐞𝐬', value=f'{likes}')
      dislikes = [x.split('d:')[-1] for x in self.attrs[self.index][2]]
      dislikes = ', '.join(dislikes)
      embed.add_field(name='𝐃𝐢𝐬𝐥𝐢𝐤𝐞𝐬', value=f'{dislikes}')
      embed.add_field(name='like count', value=f'{self.ll[self.index]}')
      embed.add_field(name='dislike count', value=f'{self.dd[self.index]}')
      await self.message.edit(embed=embed, view=self)


    @discord.ui.button(label='←', style=discord.ButtonStyle.primary)
    async def before(self, interaction:discord.Interaction,button:discord.ui.Button):
      await interaction.response.defer()
      self.index -= 1
      if self.index < 0:
        self.index = len(self.members)-1
      await self.update()

    @discord.ui.button(label='→', style=discord.ButtonStyle.primary)
    async def next(self, interaction:discord.Interaction,button:discord.ui.Button):
      await interaction.response.defer()
      self.index += 1
      self.index = self.index % len(self.members)
      await self.update()

    @discord.ui.button(label='♡', style=discord.ButtonStyle.success)
    async def like(self, interaction:discord.Interaction,button:discord.ui.Button):
      await interaction.response.defer()
      if not self.liked[self.index]:
        self.ll[self.index] += 1
        self.liked[self.index] = True
      await self.update()


    @discord.ui.button(label='✖', style=discord.ButtonStyle.danger)
    async def dislike(self, interaction:discord.Interaction,button:discord.ui.Button):
      await interaction.response.defer()
      if not self.disliked[self.index]:
        self.dd[self.index] += 1
        self.disliked[self.index] = True
      await self.update()




  @client.command()
  async def menu(ctx):

    view  = Angel()
    await view.send(ctx)





  @client.command()
  async def answer(ctx):
    await ctx.send("hello")

  @client.command()
  async def animation(interaction, seconds:int):
    '''
    await asyncio.sleep(1)
      msg = await interaction.edit_original_response(content='웃    ❤    유')
      await asyncio.sleep(1)
      msg = await interaction.edit_original_response(content='  웃  ❤  유  ')
      await asyncio.sleep(1)
      msg = await interaction.edit_original_response(content='    웃❤유    ')
    '''
    t_end = time.time() + seconds
    while time.time() < t_end:
      msg = await interaction.channel.send('웃    ❤    유')
      await asyncio.sleep(0.5)
      await msg.delete()
      msg = await interaction.channel.send('  웃  ❤  유  ')
      await asyncio.sleep(0.5)
      await msg.delete()
      msg = await interaction.channel.send('    웃❤유    ')
      await asyncio.sleep(0.5)
      await msg.delete()



  @client.command()
  async def sync(ctx):
    print("sync command")
    await client.tree.sync()
    await ctx.send('Command tree synced.')


  @client.tree.command(name="mymatches", description="find the people you match with")
  async def mymatches(interaction:discord.Interaction, gender:typing.Literal['male', 'female', 'any']):

    preferences = [gender]
    matches, per = m2.Find_Clusters(interaction, preferences)

    embed = discord.Embed(title='Your Matches On the Server',
                          description='Using our algorithm these are your results for an optimal match',
                          color = discord.Color.pink())
    await interaction.response.defer(ephemeral=True, thinking=False)

    # await interaction.followup.send('*✭˚･ﾟ✧*･ﾟ*✭˚･ﾟ✧*･ﾟ* *✭˚･ﾟ✧*･ﾟ*✭˚･ﾟ✧*･ﾟ* *✭˚･ﾟ✧*･ﾟ*✭˚･ﾟ✧*･ﾟ*')

    c = client.get_command('animation')
    await c(interaction, 6)


    if matches:
      out = ''
      # out += '*✭˚･ﾟ✧*･ﾟ*✭˚･ﾟ✧*･ﾟ* *✭˚･ﾟ✧*･ﾟ*✭˚･ﾟ✧*･ﾟ* *✭˚･ﾟ✧*･ﾟ*✭˚･ﾟ✧*･ﾟ*\n'
      for index, m in enumerate(matches):
        out += f"**Match {index+1}**\nYour one of your best matches on the server is {m.global_name} with a compatibility of {per[index]}%\nMeet your match {m.mention}"
        out += '\n\n'

      embed.add_field(name='Matches 웃❤유', value=out)
      embed.set_thumbnail(url='https://storage.googleapis.com/pod_public/1300/150289.jpg')
      await interaction.followup.send(embed=embed)

    else:
      embed.add_field(name='Matches', value="You have no matches good matches, try agian later")
      await interaction.followup.send(embed=embed)





  client.run()


run()

NameError: name 'commands' is not defined

In [ ]:
n = [1, 2, 3, 4]
m = [1, 3, 3, 3]
l = [4 ,4 ,4, 4]

In [ ]:
list(zip(l, list(zip(n,m))))

[(4, (1, 1)), (4, (2, 3)), (4, (3, 3)), (4, (4, 3))]

In [ ]:
g = ['l:a', 'l:aaaa']

In [ ]:
g = [x.split('l:')[-1] for x in g]

In [ ]:
g = [0 for x in range(2)]

In [ ]:
 python3 --version

[0, 0]